In [112]:
from __future__ import print_function

from time import time
import logging
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.datasets import fetch_lfw_people
from sklearn.decomposition import PCA as RandomizedPCA
from keras.applications.inception_v3 import InceptionV3
from keras.preprocessing import image
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D
from keras import backend as K
from keras.preprocessing.image import ImageDataGenerator
from keras.utils.np_utils import to_categorical


print(__doc__)

# Display progress logs on stdout
logging.basicConfig(level=logging.INFO, format='%(asctime)s %(message)s')


###############################################################################
# Download the data, if not already on disk and load it as numpy arrays

lfw_people = fetch_lfw_people(min_faces_per_person=70, resize=1.05)

# introspect the images arrays to find the shapes (for plotting)
n_samples, h, w = lfw_people.images.shape

print(lfw_people.images.shape)
# for machine learning we use the 2 data directly (as relative pixel
# positions info is ignored by this model)
X = lfw_people.data
n_features = X.shape[1]

# the label to predict is the id of the person
y = lfw_people.target

target_names = lfw_people.target_names
n_classes = target_names.shape[0]

print("Total dataset size:")
print("n_samples: %d" % n_samples)
print("n_features: %d" % n_features)
print("n_classes: %d" % n_classes)

Automatically created module for IPython interactive environment
(1288, 131, 98)
Total dataset size:
n_samples: 1288
n_features: 12838
n_classes: 7


split into a training and testing set

In [113]:
X = X.reshape(1288, 131, 98)
X = np.stack((X,)*3, axis=-1)
y = to_categorical(y, num_classes=n_classes)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.25)


In [114]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(966, 131, 98, 3)
(966, 7)
(322, 131, 98, 3)
(322, 7)


Compute a PCA (eigenfaces) on the face dataset (treated as unlabeled
dataset): unsupervised feature extraction / dimensionality reduction

In [115]:
# n_components = 150

# print("Extracting the top %d eigenfaces from %d faces"
#       % (n_components, X_train.shape[0]))
# t0 = time()
# pca = RandomizedPCA(n_components=n_components, whiten=True).fit(X_train)
# print("done in %0.3fs" % (time() - t0))

# eigenfaces = pca.components_.reshape((n_components, h, w, 1))
# print(eigenfaces.shape)

make model

In [118]:
# create the base pre-trained model
base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(131,98,3))

# add a global spatial average pooling layer
x = base_model.output
x = GlobalAveragePooling2D()(x)
# let's add a fully-connected layer
x = Dense(1024, activation='relu')(x)
# and a logistic layer -- let's say we have 200 classes
predictions = Dense(7, activation='softmax')(x)

# this is the model we will train
model = Model(inputs=base_model.input, outputs=predictions)

# first: train only the top layers (which were randomly initialized)
# i.e. freeze all convolutional InceptionV3 layers
for layer in base_model.layers:
    layer.trainable = False
    
# compile the model (should be done *after* setting layers to non-trainable)
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')

EPOCHS = 5
BS = 32

# train the model on the new data for a few epochs
model.fit(X_train, y_train, batch_size=None, steps_per_epoch=(len(X_train) // BS), epochs=EPOCHS)

# at this point, the top layers are well trained and we can start fine-tuning
# convolutional layers from inception V3. We will freeze the bottom N layers
# and train the remaining top layers.

# let's visualize layer names and layer indices to see how many layers
# we should freeze:
for i, layer in enumerate(base_model.layers):
   print(i, layer.name)

# we chose to train the top 2 inception blocks, i.e. we will freeze
# the first 249 layers and unfreeze the rest:
for layer in model.layers[:249]:
   layer.trainable = False
for layer in model.layers[249:]:
   layer.trainable = True

# we need to recompile the model for these modifications to take effect
# we use SGD with a low learning rate
from keras.optimizers import SGD
model.compile(optimizer=SGD(lr=0.0001, momentum=0.9), loss='categorical_crossentropy')

# we train our model again (this time fine-tuning the top 2 inception blocks
# alongside the top Dense layers
model.fit(X_train, y_train, batch_size=None, steps_per_epoch=(len(X_train) // BS), epochs=EPOCHS)

Epoch 1/5
30/30 [==============================] - 320s 11s/step - loss: 1.7194
Epoch 2/5
30/30 [==============================] - 245s 8s/step - loss: 0.7504
Epoch 3/5
30/30 [==============================] - 244s 8s/step - loss: 0.3958
Epoch 4/5
30/30 [==============================] - 255s 8s/step - loss: 0.2355
Epoch 5/5
30/30 [==============================] - 246s 8s/step - loss: 0.1926
0 input_26
1 conv2d_2223
2 batch_normalization_2223
3 activation_2221
4 conv2d_2224
5 batch_normalization_2224
6 activation_2222
7 conv2d_2225
8 batch_normalization_2225
9 activation_2223
10 max_pooling2d_98
11 conv2d_2226
12 batch_normalization_2226
13 activation_2224
14 conv2d_2227
15 batch_normalization_2227
16 activation_2225
17 max_pooling2d_99
18 conv2d_2231
19 batch_normalization_2231
20 activation_2229
21 conv2d_2229
22 conv2d_2232
23 batch_normalization_2229
24 batch_normalization_2232
25 activation_2227
26 activation_2230
27 average_pooling2d_214
28 conv2d_2228
29 conv2d_2230
30 conv2d_2

KeyboardInterrupt: 